In [1]:
import scvi
import os
import tempfile
import scanpy as sc
import scarches as sca
import seaborn as sns
import torch
import pandas as pd
from rich import print
import numpy as np
import matplotlib.pyplot as plt
import skmisc

/home/users/allstaff/moore.z/.conda/envs/scvi/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 captum (see https://github.com/pytorch/captum).


In [2]:
scvi.settings.seed = 42
torch.set_float32_matmul_precision('high')

[rank: 0] Seed set to 42


In [3]:
adata = sc.read("/vast/scratch/users/moore.z/pdo_pdn_comparison/data/processed/2_5_tumour_only/GL0128_tumour.h5ad")
adata

AnnData object with n_obs × n_vars = 550 × 19463
    obs: 'bc_wells', 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'bc1_wind', 'bc2_wind', 'bc3_wind', 'bc1_well', 'bc2_well', 'bc3_well', 'doublet_score', 'predicted_doublets', 'sum', 'detected', 'subsets_mito_sum', 'subsets_mito_detected', 'subsets_mito_percent', 'total', 'low_lib_size', 'low_n_features', 'high_subsets_mito_percent', 'discard', 'discard_mito', 'discard_feature', 'discard_final', 'sizeFactor', 'leiden_individual'
    var: 'gene_id', 'gene_name', 'genome', 'gene_version', 'gene_source', 'gene_biotype'
    uns: 'X_name'
    layers: 'logcounts'

In [4]:
sc.pp.highly_variable_genes(
    adata = adata, 
    flavor = "seurat_v3",
    n_top_genes = 5000, 
    subset = True
)

In [5]:
# integration with scVI
scvi.model.SCVI.setup_anndata(
    adata = adata, 
    batch_key = "sample"
)
# create model
scvi_model = scvi.model.SCVI(
    adata = adata,
    n_hidden = 128,
    n_latent = 50,
    dispersion = "gene-batch"
)
# train model
# scvi.settings.dl_num_workers = 15
scvi_model.train(
    max_epochs = 1000, 
    early_stopping = True
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.
/home/users/allstaff/moore.z/.conda/envs/scvi/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/users/allstaff/moore.z/.conda/envs/scvi/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
/home/users/allstaff/moore.z/.conda/envs/scvi/lib/python3.11/site-packages/lightning/pytorch/trainer/conne

Epoch 223/1000:  22%|██▏       | 223/1000 [00:08<00:30, 25.17it/s, v_num=1, train_loss_step=2.33e+3, train_loss_epoch=2.39e+3]
Monitored metric elbo_validation did not improve in the last 45 records. Best score: 2699.300. Signaling Trainer to stop.


In [ ]:
train_test_results = scvi_model.history["elbo_train"]
train_test_results["elbo_validation"] = scvi_model.history["elbo_validation"]
train_test_results.iloc[10:].plot(logy = True)  # exclude first 10 epochs
plt.show()

In [6]:
SCVI_LATENT_KEY = "X_scVI"
adata.obsm[SCVI_LATENT_KEY] = scvi_model.get_latent_representation()

In [7]:
sc.pp.neighbors(
    adata = adata, 
    use_rep = SCVI_LATENT_KEY, 
    n_neighbors = 15,
    n_pcs = 50,
    method = "gauss",
    knn = False
)
# sc.tl.umap(
#     adata = adata, 
#     min_dist = 0.005
# )

In [8]:
sc.tl.leiden(
    adata,
    flavor = "igraph",
    resolution = 1.25,
    # n_iterations = 10
)

In [9]:
SCVI_MDE_KEY = "X_scVI_MDE"
adata.obsm[SCVI_MDE_KEY] = scvi.model.utils.mde(adata.obsm[SCVI_LATENT_KEY])

INFO     Using cuda:0 for `pymde.preserve_neighbors`.                                                              


In [ ]:
sc.pl.embedding(
    adata = adata,
    basis = SCVI_MDE_KEY,
    color = ["sample", "leiden"],
    # legend_loc = "on data",
    frameon = False,
    ncols = 2
)

In [12]:
adata.write_h5ad("/vast/scratch/users/moore.z/pdo_pdn_comparison/data/processed/2_5_tumour_only/GL0128_tumour_scvi.h5ad")

In [13]:
change_per_cluster = scvi_model.differential_expression(
    adata = adata, 
    groupby = "leiden", 
    batch_correction = True,
    filter_outlier_cells = False
)

DE...: 100%|██████████| 7/7 [00:07<00:00,  1.06s/it]


In [15]:
change_per_cluster.to_csv(path_or_buf = "/vast/scratch/users/moore.z/pdo_pdn_comparison/data/processed/2_5_tumour_only/GL0128_change_per_clust.csv")

In [16]:
import scipy as sp
con = adata.obsp["connectivities"]
dist = adata.obsp["distances"]

In [17]:
con = pd.DataFrame(con)
dist = pd.DataFrame(dist)

In [18]:
con.to_csv("/vast/scratch/users/moore.z/pdo_pdn_comparison/data/processed/2_5_tumour_only/GL0128_con.csv")
dist.to_csv("/vast/scratch/users/moore.z/pdo_pdn_comparison/data/processed/2_5_tumour_only/GL0128_dist.csv")